In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import functions as fn
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_CCM = pd.read_csv('data/ccm_manipulated_347418.csv', names=['EVENT_ID', 'SEQUENCE_ID', 'R_OS_ID', 'PRODUCTION_LINE_NAME', 'REFERENCE_DATE', 'PIECE_ID', 
 'MATERIAL_ID', 'MOLD_WIDTH', 'WIDTH', 'THICKNESS', 'WEIGHT', 'LENGTH', 'HEAT_ID',
 'STEEL_GRADE_ID_INT', 'EXIT_TEMP', 'SLAB_TRANSITION'], header=None)
df_CCM

,EVENT_ID,SEQUENCE_ID,R_OS_ID,PRODUCTION_LINE_NAME,REFERENCE_DATE,PIECE_ID,MATERIAL_ID,MOLD_WIDTH,WIDTH,THICKNESS,WEIGHT,LENGTH,HEAT_ID,STEEL_GRADE_ID_INT,EXIT_TEMP,SLAB_TRANSITION
0,8360,109,115587,CCM1,13.12.16,16000041-07,148513,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
1,8359,109,115587,CCM1,13.12.16,16000041-06,148512,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
2,8358,109,115587,CCM1,13.12.16,16000041-05,148511,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
3,8357,109,115587,CCM1,13.12.16,16000041-03,148509,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
4,8356,109,115587,CCM1,13.12.16,16000041-02,148508,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347413,458481,4154,891748,CCM1,13.02.18,18023901-07,352574,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
347414,458480,4154,891748,CCM1,13.02.18,18023901-06,352573,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
347415,458479,4154,891748,CCM1,13.02.18,18023901-05,352572,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
347416,458478,4154,891748,CCM1,13.02.18,18023901-04,352571,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0


In [3]:
feature = 8 # Width
aim = df[feature]
min_val = int(aim.min())
max_val = int(aim.max())
print(f"Min: {min_val}, Max: {max_val}")
step = 11
range_values = list(range(min_val, max_val + step, step))

# Create bin intervals (Partition)
bins = [(range_values[i], range_values[i+1]) 
    for i in range(len(range_values)-1)]
bins

Min: 915, Max: 1920


[(915, 926),
 (926, 937),
 (937, 948),
 (948, 959),
 (959, 970),
 (970, 981),
 (981, 992),
 (992, 1003),
 (1003, 1014),
 (1014, 1025),
 (1025, 1036),
 (1036, 1047),
 (1047, 1058),
 (1058, 1069),
 (1069, 1080),
 (1080, 1091),
 (1091, 1102),
 (1102, 1113),
 (1113, 1124),
 (1124, 1135),
 (1135, 1146),
 (1146, 1157),
 (1157, 1168),
 (1168, 1179),
 (1179, 1190),
 (1190, 1201),
 (1201, 1212),
 (1212, 1223),
 (1223, 1234),
 (1234, 1245),
 (1245, 1256),
 (1256, 1267),
 (1267, 1278),
 (1278, 1289),
 (1289, 1300),
 (1300, 1311),
 (1311, 1322),
 (1322, 1333),
 (1333, 1344),
 (1344, 1355),
 (1355, 1366),
 (1366, 1377),
 (1377, 1388),
 (1388, 1399),
 (1399, 1410),
 (1410, 1421),
 (1421, 1432),
 (1432, 1443),
 (1443, 1454),
 (1454, 1465),
 (1465, 1476),
 (1476, 1487),
 (1487, 1498),
 (1498, 1509),
 (1509, 1520),
 (1520, 1531),
 (1531, 1542),
 (1542, 1553),
 (1553, 1564),
 (1564, 1575),
 (1575, 1586),
 (1586, 1597),
 (1597, 1608),
 (1608, 1619),
 (1619, 1630),
 (1630, 1641),
 (1641, 1652),
 (1652, 16

In [7]:
# For each value in aim, find its bin
binning = []
for value in aim:
    # Find first bin that contains this value (Catch/Throw)
    for bin_interval in bins:
        if bin_interval[0] <= value <= bin_interval[1]:
            binning.append(bin_interval)
            break  # Equivalent to Throw
    else:
        binning.append(None)  # No bin found
print(len(binning))
binning

347418


[(1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1289, 1300),
 (1289, 1300),
 (1289, 1300),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 12

In [5]:
def binning_dataframe(df, value_column, min_val, max_val, step):
    """
    DataFrame-friendly version for ETL pipelines.
    Perfect for data engineering workflows.
    """
    result_df = df.copy()
    
    # Create bin edges
    edges = np.arange(min_val, max_val + step, step)
    intervals = pd.IntervalIndex.from_breaks(edges, closed='both')
    
    # Find bin for each value
    def find_bin(value):
        for interval in intervals:
            if value in interval:
                return (int(interval.left), int(interval.right))
        return None
    
    result_df['bin_interval'] = result_df[value_column].apply(find_bin)
    result_df['bin_left'] = result_df['bin_interval'].apply(lambda x: x[0] if x else None)
    result_df['bin_right'] = result_df['bin_interval'].apply(lambda x: x[1] if x else None)
    
    return result_df

binning_dataframe(df, 8, int(df[8].min()), int(df[8].max()), 11)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,bin_interval,bin_left,bin_right
0,8360,109,115587,CCM1,13.12.16,16000041-07,148513,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0,"(1322, 1333)",1322,1333
1,8359,109,115587,CCM1,13.12.16,16000041-06,148512,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0,"(1322, 1333)",1322,1333
2,8358,109,115587,CCM1,13.12.16,16000041-05,148511,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0,"(1322, 1333)",1322,1333
3,8357,109,115587,CCM1,13.12.16,16000041-03,148509,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0,"(1322, 1333)",1322,1333
4,8356,109,115587,CCM1,13.12.16,16000041-02,148508,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0,"(1322, 1333)",1322,1333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347413,458481,4154,891748,CCM1,13.02.18,18023901-07,352574,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0,"(1520, 1531)",1520,1531
347414,458480,4154,891748,CCM1,13.02.18,18023901-06,352573,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0,"(1520, 1531)",1520,1531
347415,458479,4154,891748,CCM1,13.02.18,18023901-05,352572,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0,"(1520, 1531)",1520,1531
347416,458478,4154,891748,CCM1,13.02.18,18023901-04,352571,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0,"(1520, 1531)",1520,1531


In [3]:
def mathematica_binning(aim, min_val, max_val, step):
    """
    Python equivalent of Mathematica's binning code using pandas.
    
    Replicates:
    binning=Table[Catch[Do[If[IntervalMemberQ[Interval[i],aim[[j]]]==True,Throw[i]],
    {i,Partition[Range[min,max,step],2,1]}]],{j,Length[aim]}];
    
    Parameters:
    -----------
    aim : list or array
        Values to bin
    min_val : numeric
        Minimum bin boundary
    max_val : numeric
        Maximum bin boundary
    step : numeric
        Bin width
        
    Returns:
    --------
    list of tuples : Bin intervals (left, right) for each value
    """
    # Create bin edges
    edges = np.arange(min_val, max_val + step, step)
    
    # Create interval index with closed boundaries on both sides
    intervals = pd.IntervalIndex.from_breaks(edges, closed='both')
    
    # Find matching bin for each value
    result = []
    for value in aim:
        matching = [i for i in intervals if value in i]
        if matching:
            interval = matching[0]
            result.append((int(interval.left), int(interval.right)))
        else:
            result.append(None)
    
    return result

mathematica_binning(df[8], int(df[8].min()), int(df[8].max()), 11)

[(1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1289, 1300),
 (1289, 1300),
 (1289, 1300),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 12

In [ ]:
class BinningOptimized:
    """
    Pre-computes intervals for reuse in data pipelines.
    Best for processing multiple batches with same bin structure.
    """
    def __init__(self, min_val, max_val, step):
        edges = np.arange(min_val, max_val + step, step)
        self.intervals = pd.IntervalIndex.from_breaks(edges, closed='both')
        
    def bin(self, values):
        result = []
        for value in values:
            matching = [i for i in self.intervals if value in i]
            if matching:
                interval = matching[0]
                result.append((int(interval.left), int(interval.right)))
            else:
                result.append(None)
        return result

# Usage
binner = BinningOptimized(int(df[8].min()), int(df[8].max()), 11)
result1 = binner.bin([1.5, 3.2])  # Reuses pre-computed intervals
result2 = binner.bin([7.8, 9.1])

In [6]:
def binning_pandas_cut(aim, min_val, max_val, step):
    """Using pd.cut() for simpler binning."""
    bins = np.arange(min_val, max_val + step, step)
    binned = pd.cut(aim, bins=bins, include_lowest=True)
    result = [(int(interval.left), int(interval.right)) for interval in binned]
    return result

binning_pandas_cut(df[8], int(df[8].min()), int(df[8].max()), 11)

[(1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1322, 1333),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1245, 1256),
 (1289, 1300),
 (1289, 1300),
 (1289, 1300),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 1245),
 (1234, 12